In [1]:
import os
from pathlib import Path
import sys

PATH_PROJECT_ROOT_DIR = Path(os.getcwd()).resolve().parent
PATH_SRC_DIR = os.path.join(PATH_PROJECT_ROOT_DIR, "src/")

sys.path.append(str(PATH_SRC_DIR))

In [9]:
os.getenv("CONSUMER_KEY")
os.environ["REQUESTS_CA_BUNDLE"] = "/etc/pki/tls/certs/sanofi-ca.crt"


In [3]:
import os
# Ensure corporate CA bundle used for SSL verification
ca_bundle = '/etc/pki/tls/certs/sanofi-ca.crt'
os.environ['SSL_CERT_FILE'] = ca_bundle

import tweepy
import pandas as pd
from transformers import pipeline
# import yfinance as yf
import matplotlib.pyplot as plt

# Load environment variable for bearer token
bearer_token = os.getenv('BEARER_TOKEN')
if not bearer_token:
    raise RuntimeError('Missing TWITTER_BEARER_TOKEN environment variable.')

# Initialize Tweepy client for API v2
def create_client(verify_ssl=True):
    client = tweepy.Client(bearer_token=bearer_token)
    # Override SSL verification if desired
    client.session.verify = ca_bundle if verify_ssl else False
    return client

# Function to check if bearer token itself is valid, bypassing SSL issues
def check_token_validity():
    # Create client without SSL validation temporarily
    test_client = create_client(verify_ssl=False)
    try:
        # Attempt to fetch a known public user
        test_client.get_user(username='TwitterDev')
        print('✅ Credentials appear valid (received response without SSL).')
        return True
    except tweepy.Unauthorized:
        print('❌ Unauthorized: Bearer token is invalid or lacks access rights.')
        return False
    except Exception as e:
        print(f'⚠️ Error during token check (likely SSL related): {e}')
        return None

# Check credentials validity
valid = check_token_validity()
if valid is False:
    raise RuntimeError('Bearer token invalid. Please verify your credentials.')
elif valid is None:
    print('Proceeding with SSL-enabled client; SSL issues may persist.')

# Initialize client with SSL verification enabled
client = create_client(verify_ssl=True)

# List of influential accounts\accounts = ['zerohedge', 'business', 'TheStalwart', 'ReutersBiz', 'CNBC', 'FT', 'WSJ']

# Fetch recent tweets for a user
def fetch_user_tweets(user_handle, max_results=5):
    try:
        user = client.get_user(username=user_handle)
        user_id = user.data.id
        tweets = client.get_users_tweets(id=user_id, max_results=max_results)
        return [t.text for t in (tweets.data or [])]
    except tweepy.Unauthorized:
        print(f'❌ Unauthorized access for @{user_handle}. Token may lack permissions.')
    except tweepy.TooManyRequests:
        print(f'⚠️ Rate limit reached for @{user_handle}, skipping...')
    except Exception as e:
        print(f'Error fetching tweets for @{user_handle}: {e}')
    return []

# Collect tweets
tweet_list = []
for user in accounts:
    tweet_list = fetch_user_tweets(user, max_results=5)
    tweet_list.extend(tweet_list)

if not tweet_list:
    raise RuntimeError('No tweets collected. Possible bad token or SSL issues.')

# Sentiment analysis pipeline
sentiment_model = pipeline('sentiment-analysis', model='finiteautomata/bertweet-base-sentiment-analysis')

# Analyze and score tweets
sentiment_scores = [sentiment_model(text)[0]['label'] for text in tweet_list]
sentiment_df = pd.DataFrame({'Tweet': tweet_list, 'Sentiment': sentiment_scores})
sentiment_df['Score'] = sentiment_df['Sentiment'].apply(lambda x: 1 if x == 'POSITIVE' else -1)

# Aggregate sentiment and fetch VIX data
sentiment_score = sentiment_df['Score'].sum()
vix_data = yf.download('^VIX', period='1mo', interval='1d')

# Trading signal
gsignal = 'BUY' if sentiment_score < -10 else 'HOLD'
print('Aggregated Sentiment Score:', sentiment_score)
print('Trading Signal:', gsignal)

# Plot VIX closing price
vix_data['Close'].plot(title='VIX Price Over Time', figsize=(12, 6))
plt.show()


/home/u1017597/miniconda3/envs/financial_nlp_env/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.twitter.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ Credentials appear valid (received response without SSL).


NameError: name 'accounts' is not defined

In [ ]:
import os
import tweepy
import pandas as pd
from transformers import pipeline
# import yfinance as yf
import matplotlib.pyplot as plt
os.environ['SSL_CERT_FILE'] = '/etc/pki/tls/certs/sanofi-ca.crt'

# Load environment variables for tokens (ensure these are set in your environment)
api_key = os.getenv("API_KEY")
api_secret = os.getenv("API_SECRET")
bearer_token = os.getenv("BEARER_TOKEN")

# Initialize Tweepy client for API v2
client = tweepy.Client(bearer_token=bearer_token)

# Define influential financial accounts to query
accounts = [
    'zerohedge'
]

# Utility: get tweets for one user using free access
def fetch_user_tweets(user_handle, max_results=5):
    try:
        # Retrieve user ID from username
        user = client.get_user(username=user_handle)
        user_id = user.data.id
        # Fetch the most recent tweets by ID (free: 1 request per 15 mins)
        tweets = client.get_users_tweets(id=user_id, max_results=max_results)
        return [t.text for t in (tweets.data or [])]
    except tweepy.TooManyRequests:
        print(f"Rate limit reached for @{user_handle}, skipping...")
    except Exception as e:
        print(f"Error fetching tweets for @{user_handle}: {e}")
    return []

# Collect tweets from each account
tweet_list = []
for user in accounts:
    tweets = fetch_user_tweets(user, max_results=5)
    tweet_list.extend(tweets)

if not tweet_list:
    raise RuntimeError("No tweets collected; check API access level or credentials.")

# Load pretrained sentiment analysis pipeline
sentiment_model = pipeline(
    "sentiment-analysis", model="finiteautomata/bertweet-base-sentiment-analysis"
)

# Apply sentiment analysis
sentiment_scores = [sentiment_model(text)[0]['label'] for text in tweet_list]
sentiment_df = pd.DataFrame({'Tweet': tweet_list, 'Sentiment': sentiment_scores})

# Numeric scoring: +1 for positive, -1 for negative/neutral
sentiment_df['Score'] = sentiment_df['Sentiment'].apply(lambda x: 1 if x == 'POSITIVE' else -1)

# Aggregate sentiment
sentiment_score = sentiment_df['Score'].sum()

# Fetch VIX data
vix_data = yf.download('^VIX', period='1mo', interval='1d')

# Generate trading signal
signal = 'BUY' if sentiment_score < -10 else 'HOLD'

print('Aggregated Sentiment Score:', sentiment_score)
print('Trading Signal:', signal)

# Plot VIX closing price for visualization
vix_data['Close'].plot(title='VIX Price Over Time', figsize=(12, 6))
plt.show()


Error fetching tweets for @zerohedge: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/users/by/username/zerohedge (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1016)')))
Error fetching tweets for @business: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/users/by/username/business (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1016)')))
Error fetching tweets for @TheStalwart: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/users/by/username/TheStalwart (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1016)')))
Error fetching tw

RuntimeError: No tweets collected; check API access level or credentials.

In [4]:
import tweepy

client = tweepy.Client(bearer_token=os.getenv("BEARER_TOKEN"))
response = client.search_recent_tweets("Tweepy")



SSLError: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/tweets/search/recent?query=Tweepy (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1016)')))

In [8]:
import tweepy

client = tweepy.Client(bearer_token="TU_BEARER_TOKEN")

query = "VIX OR SP500"
response = client.search_recent_tweets(query=query, max_results=2)

for tweet in response.data:
    print(tweet.text)


SSLError: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/tweets/search/recent?max_results=2&query=VIX+OR+SP500 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1016)')))

In [5]:
tweets = api.search_tweets(q="VIX OR SP500", lang="en", count=2)
for tweet in tweets:
    print(tweet.user.screen_name, "→", tweet.text)


NameError: name 'api' is not defined

In [7]:
response = client.search_recent_tweets(query="VIX OR SP500", max_results=2)
response

SSLError: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /2/tweets/search/recent?max_results=2&query=VIX+OR+SP500 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1016)')))